# PROJECT 1: EXPLORATORY DATA ANALYSIS

## Project Name: 

SOUNDCLOUD ANALYSIS

## Assignment Table

| ID            | Fullname              | Contribution rate |
| :-----------: | :-------------------  | :-----------:     |
| 20127323      | Võ Nhật Tân           |                   |
| 20127447      | Ngô Đức Bảo           |                   |
| 20127275      | Lê Nguyễn Nhật Phú    |                   |
| 20127681      | Nguyễn Thiên Phúc     |                   |



## Data Collection

### Collect Users

In [11]:
import time
import pandas as pd
import random
import requests
import requests_cache
from bs4 import BeautifulSoup

In [12]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [13]:
sleep_time = 2

In [14]:
SCROLL_PAUSE_TIME = 0.5

In [15]:
requests_cache.install_cache('cache', expire_after=None)

In [27]:
urlList = [ 'https://soundcloud.com/user-593335594/sets/is-500-songs-the-playlist', \
            'https://soundcloud.com/user-593335594/sets/amazing-artistry-2',\
            'https://soundcloud.com/user-593335594/sets/playlist-that-wont-crash-my',\
            'https://soundcloud.com/user-593335594/sets/my-personal-favorites',\
            'https://soundcloud.com/user-593335594/sets/amazing',\
            'https://soundcloud.com/user-593335594/sets/i-found-more-music',\
            'https://soundcloud.com/user-593335594/sets/chivalry-is-dead',\
            'https://soundcloud.com/idla/sets/old-songs',\
            'https://soundcloud.com/idla/sets/pop-playlist-2020',\
            'https://soundcloud.com/idla/sets/lost-in-the-blues',\
            'https://soundcloud.com/user987150052/sets/hot-playlist',\
            'https://soundcloud.com/user61185041/sets/hot-playlist',\
            'https://soundcloud.com/discover/sets/charts-top:all-music:vn',\
            'https://soundcloud.com/discover/sets/charts-trending:danceedm:vn',\
            'https://soundcloud.com/bytufekci/sets/top-100-songs-of-2020']

In [24]:
def getUserUrl(urlList):
    driver = webdriver.Chrome()
    userUrl = set()
    isCheckCookie = True
    for url in urlList:
        driver.get(url)
        driver.maximize_window()
        time.sleep(sleep_time)
        if isCheckCookie: 
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))).click()
            isCheckCookie = False
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)
            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'trackItem__username')))
        newUserUrl = driver.find_elements(By.CLASS_NAME, 'trackItem__username')
        for i in newUserUrl:
            userUrl.add(i.get_attribute('href'))
        print("Total of user's url",len(userUrl))
    driver.close()
    return userUrl


In [25]:
userUrl = getUserUrl(urlList)

Total of user's url 318
Total of user's url 325
Total of user's url 374
Total of user's url 386
Total of user's url 580
Total of user's url 604
Total of user's url 614
Total of user's url 693
Total of user's url 752
Total of user's url 798
Total of user's url 1003
Total of user's url 1279


In [46]:
#create a list of users using dataframes
users = pd.DataFrame(columns=['userID', 'username', 'userUrl', 'followers', 'following', 'tracks', 'playlistIDs'])

In [47]:
userUrlList = list(userUrl)
print(userUrlList[0])

https://soundcloud.com/ambition-beats


In [48]:
def get_user_info(userUrl):    
    driver = webdriver.Chrome()
    isCheckCookie = True
    for url in userUrl:
        driver.get(url)
        driver.maximize_window()
        time.sleep(sleep_time)
        if isCheckCookie: 
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))).click()
            isCheckCookie = False
        username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'profileHeaderInfo__userName'))).text
        userID = url.split('/')[-1]
        userurl = url
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'infoStats__value.sc-font-light')))
        info = driver.find_elements(By.CLASS_NAME, 'infoStats__value.sc-font-light')
        followers = info[0].text
        following = info[1].text
        tracks = info[2].text
        #go to user's sets which have the url https://soundcloud.com/{userID}/sets
        driver.get(f'https://soundcloud.com/{userID}/sets')
        time.sleep(sleep_time)
        playlist = driver.find_elements(By.CLASS_NAME, 'sc-link-primary.soundTitle__title.sc-link-dark.sc-text-h4')
        playlistIDs = []
        if len(playlist) == 0:
            playlistIDs = None
        else:
            for p in playlist:
                playlistIDs.append(p.get_attribute('href').split('/')[-1])
        users.loc[len(users)] = [userID, username, userurl, followers, following, tracks, playlistIDs]
    driver.close()

In [49]:
#this cell takes a long time to run (more than 114 minutes to go through 1278 users)
for i in range(0, len(userUrlList), 200):
    start = i
    end = i + 200
    if end > len(userUrlList):
        end = len(userUrlList)
    get_user_info(userUrlList[start:end])
    users.to_csv('users' + str(i) + '.csv', index=False)
    print(users.shape)

(200, 7)
(400, 7)
(600, 7)
(800, 7)
(1000, 7)
(1200, 7)
(1279, 7)


In [50]:
print(users)
users.to_csv('users.csv', index=False)

                                userID                          username  \
0                       ambition-beats                 Ambition Verified   
1                   downchildbluesband              Downchild Blues Band   
2                            blindface                         BlindFace   
3                             leiticia                           Letícia   
4                              adamazz                            NoCtrl   
...                                ...                               ...   
1274                          maroon-5                 Maroon 5 Verified   
1275  five-finger-death-punch-official  Five Finger Death Punch Verified   
1276                    allaboutmaggie                      Maggie Szabo   
1277                        en_network           EN - NETWORK™️ Verified   
1278              david-clayton-thomas              David Clayton-Thomas   

                                                userUrl followers following  \
0       

### Collect Playlists

### Collect Tracks

## Exploratory Data Analysis

## References